In [1]:
!pip install chromadb langchain langchain_community

Defaulting to user installation because normal site-packages is not writeable


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
! pip install langchain_text_splitters pydantic langchain-ollama pypdf gradio Markdown
# openai
# langchain-openai
# python-dotenv
sentence-transformers
# transformers
# langchain-huggingface

Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 8.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
botocore 1.37.20 requires urllib3<1.27,>=1.25.4; python_version < "3.10", but you have u

In [10]:
!pip install sentence-transformers transformers langchai-huggingface

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement langchai-huggingface (from versions: none)
ERROR: No matching distribution found for langchai-huggingface

[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [21]:
!pip install sentencepiece


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import gradio as gr
import os
import re
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.llms import HuggingFaceHub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import StrOutputParser
from markdown import markdown
from transformers import pipeline

In [16]:
# Load API keys from .env file
load_dotenv()
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")

# Define Hugging Face models
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # Small, fast embeddings
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"

# Initialize Embeddings model
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

# Load LLM as a text generation pipeline
llm_pipeline = pipeline("text-generation", model=LLM_MODEL, token=hf_api_key, max_new_tokens=200, use_fast=True)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Global variables
VECTOR_DB_NAME = "huggingface-rag"
vector_db = None


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
# Function to load PDF
def load_pdf(file_path):
    print(f"Loading PDF: {file_path}")
    loader = PyPDFLoader(file_path=file_path)
    print("File loaded Sucessfully!")
    return loader.load()

# Function to split text
def split_text(data, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    print("Data SPlitted Sucessfully!")
    return text_splitter.split_documents(data)

# Function to create vector database
def create_vector_db(chunks):
    global vector_db
    if not chunks:
        return "Error: No text extracted from the PDF."
    print("Creating vector database...")
    vector_db = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_model,
        collection_name=VECTOR_DB_NAME,
        persist_directory="./chroma_db5"
    )
    return "Vector database created successfully"

# Function to set up retriever
def get_retriever():
    if not vector_db:
        print("Error: Vector database is not initialized.")
        return None

    query_prompt = PromptTemplate(
        input_variables=["question"],
        template="Generate 2 alternative versions of the question to improve retrieval:\nOriginal: {question}"
    )

    return MultiQueryRetriever.from_llm(vector_db.as_retriever(), llm, prompt=query_prompt)

# Function to create RAG chain
def create_rag_chain():
    retriever = get_retriever()
    if not retriever:
        return None

    template = "Answer based ONLY on the following context:\n{context}\nQuestion: {question}"
    prompt = ChatPromptTemplate.from_template(template)

    return (
        {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
    )

# Function to query the document
def chat_with_pdf(question):
    global vector_db
    if not vector_db:
        return "Error: No vector database found. Please upload and process a PDF first."

    chain = create_rag_chain()
    if not chain:
        return "Error: Unable to initialize the RAG chain."

    print(f"Processing question: {question}")
    response = chain.invoke(question)
    print(f"Response: {response}")

    return markdown(response)

# Gradio Interface
def gradio_interface(file):
    global vector_db

    if not file:
        return "Please upload a valid PDF file."
    
    # Reset the vector DB before adding new embeddings
    vector_db = None
    # Load and process the PDF
    file_path = file.name
    data = load_pdf(file_path)
    # chunks = split_text(data[:1])
    chunks = split_text(data)
    
    message = create_vector_db(chunks)
    
    if "Error" in message:
        return message
    
    return "PDF processed successfully! You can now ask questions."

In [18]:
# Create Gradio UI
gui = gr.Blocks()
with gui:
    gr.Markdown("## Chat with your PDF using RAG (Hugging Face)")
    file_input = gr.File(label="Upload PDF")
    process_button = gr.Button("Process PDF")
    status_output = gr.Textbox()
    question_input = gr.Textbox(label="Ask a question")
    submit_button = gr.Button("Ask")
    answer_output = gr.Markdown()
    
    process_button.click(gradio_interface, inputs=file_input, outputs=status_output)
    submit_button.click(chat_with_pdf, inputs=question_input, outputs=answer_output)

# Launch the Gradio app
gui.launch(share=True)


Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://73353f83e55f3101b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Loading PDF: /tmp/gradio/174e43898afd0be5fe10fdc18783e4b5a2e19007c19747900bf7af6dd25e9594/Academic_CV.pdf
File loaded Sucessfully!
Data SPlitted Sucessfully!
Creating vector database...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing question: Owais currently works for which organization?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response: Human: Answer based ONLY on the following context:
[Document(metadata={'page': 1, 'source': '/tmp/gradio/174e43898afd0be5fe10fdc18783e4b5a2e19007c19747900bf7af6dd25e9594/Academic_CV.pdf'}, page_content='• Managingresearchdataandensuringitsaccuracyandintegrity.\n• Participatinginresearch-relatedseminarsandworkshops.\nTechnicalExperience\nProject-2022,KhaiporaCollegeWebsite\n• Analysed and gathered business requirements specifications by interacting with client and understand\nbusinessrequirementspecificationdocuments.\n• ImplementedfrontofthewebsiteusingPython,CSSandJSON.\n• WorkedonfrontendframeworkslikeCSS,Bootstrapforresponsivewebpages.\n• UsedRestfulAPIstoaccessdatafrompages.\n• Developedbackendusingtheflaskframework.\n• CreatedschemaforMySQLDatabaseandusedSQLforperformingvariousCRUDoperations.\nProject-2021,DeeplearningbasedModelforPhysicalActivityRecognition\n• Usedconvolutionalneuralnetwork(CNN)forautomaticfeatureextractionfromthetimeseriessensordata\n(https://archive.i

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processing question: What tools have owais used in different projects?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response: Human: Answer based ONLY on the following context:
[Document(metadata={'page': 1, 'source': '/tmp/gradio/174e43898afd0be5fe10fdc18783e4b5a2e19007c19747900bf7af6dd25e9594/Academic_CV.pdf'}, page_content='• Managingresearchdataandensuringitsaccuracyandintegrity.\n• Participatinginresearch-relatedseminarsandworkshops.\nTechnicalExperience\nProject-2022,KhaiporaCollegeWebsite\n• Analysed and gathered business requirements specifications by interacting with client and understand\nbusinessrequirementspecificationdocuments.\n• ImplementedfrontofthewebsiteusingPython,CSSandJSON.\n• WorkedonfrontendframeworkslikeCSS,Bootstrapforresponsivewebpages.\n• UsedRestfulAPIstoaccessdatafrompages.\n• Developedbackendusingtheflaskframework.\n• CreatedschemaforMySQLDatabaseandusedSQLforperformingvariousCRUDoperations.\nProject-2021,DeeplearningbasedModelforPhysicalActivityRecognition\n• Usedconvolutionalneuralnetwork(CNN)forautomaticfeatureextractionfromthetimeseriessensordata\n(https://archive.i